# Calculating the Column Density and Mass of the M43 HII Region


## 1. Import Relevant Data and Packages
Here, we'll import important astropy and numpy packages, as well as the data files we'll need for M43--the reigon mapped in 12CO, 13CO, and C18O. We'll also import region files and two previously made excitiation temperature and maximum value maps, which will be important in calculating the column density.

Lastly, we also add the pre-made integrated intensity maps (made with spectral-cube package).

We reference some functions from Functions.py, which can be seen for reference in the github repository.

### Import data files and packages

In [3]:
from Functions import *
import numpy as np
from astropy.io import fits
from astropy import units as u
import math
import matplotlib.pyplot as plt
from spectral_cube import SpectralCube
from astropy.wcs import WCS

CO12 = load_fits('CARMA+NRO_OrionA_12CO_comgrid.fits')
CO13 = load_fits('CARMA+NRO_OrionA_13CO_comgrid.fits')
C18O = load_fits('CARMA+NRO_OrionA_C18O_comgrid.fits')


### Now import the maps and region files

In [4]:
#Add Other files
Tex12 = load_fits('tex12co.fits')
Max13CO = load_fits('13COMax.fits')

#Load regions
M43_Estimate_One = 'm43_estimate_ONEA.reg'
region1 = pyregion.open(M43_Estimate_One)

M43_Estimate_Two = 'm43_estimate_TWO.reg'
region2 = pyregion.open(M43_Estimate_Two)

M43_Estimate_Three = 'm43_estimate_THREE.reg'
region3 = pyregion.open(M43_Estimate_Three)

### Add the integrated intensity maps


In [5]:
#Load in 13CO IIm
CO13_IIM = load_fits('IIM_13co_M43.fits')

## 2. 13CO

We start our calculations with 13CO because the column denssity equations are most likely to work well with this tracing gas (12CO is likely too optically thick for an ideal estimate).

### A1. Column Density Method 1
We use the column density method found in Kong et. al 2019. The function can be called from Functions.py but included below for convenience.

In [6]:
def CD_Pineda_unmasked(iim, Tex, max_map):
    '''
    Function returns column density map of data cube using the method in Kong et. al
    2019 (Pineda eqn 7/9).
    
    NOTE: We assume Tex of 12Co can be used for Tex 13CO as well.
    
    !!: currently set up for 13CO data only
    
    Parameters
    ---------------
    iim: 2d array
        Integrated intensity map of data
    tex: 2d array
        Excitation temperature map of data
    max_map: 2d array
        Map of maximum values of the data
    
    Returns
    --------------
    CD_Kong:
        Column Density map of data 
        
    '''
    
    
    #Evaluate Taue using Pineda equation 7
    Tau = -np.log(1- (max_map / 5.3) / (1/(np.exp(5.3/Tex) -1) -0.16))
    
    #Now Bourke eqn 5
    
    CD = ((Tau/(1 - np.exp(-Tau))) * (3*10**14) * (iim/(1-np.exp(-5.3/Tex))))
    
    return CD

#### Bug # 1
Currently, the above function loses the WCS data, which is not ideal. This stems from the funciton accessing only the data component of the FITS Cube. It will be rewritten to append the WCS data again witin the function, but at the moment, we do that externally below


To work around the bug for the moment, we open the fits file of the cube that has already been run through the above function

In [7]:
CD_Pineda = fits.open('CD_Pineda.fits')

CD_Pineda_w_header = add_header(CD_Pineda)

### A2. Column Density Method 2
Similar to method A, but we act in the optically thin assumption (not unreasonable for 13CO). Again, function provided for convenience.

Bug #1 extends to this method as well, so same workaround is used.

In [8]:
def CD_Pineda_unmasked_opticthin(iim, Tex, max_map):
    '''
    Function returns column density map of data cube using the method in Kong et. al
    2019 (Pineda eqn 7/9). We assume optically thin and omit the leading coefficient in Pineda
    eqn. 9.
    
    NOTE: We assume Tex of 12Co can be used for Tex 13CO as well.
    
    !! currently set up for 13CO data only
    
    Parameters
    ---------------
    iim: 2d array
        Integrated intensity map of data
    tex: 2d array
        Excitation temperature map of data
    max_map: 2d array
        Map of maximum values of the data
    
    Returns
    --------------
    CD_Kong:
        Column Density map of data 
        
    '''
        
    #Evaluate Taue using Pineda equation 7
    Tau = -np.log(1- (max_map / 5.3) / (1/(np.exp(5.3/Tex) -1) -0.16))
    
    #Now Bourke eqn 5
    
    CD = (3*10**14) * (iim/(1-np.exp(-5.3/Tex)))
    
    return CD

In [9]:
CD_Pineda_opticthin = fits.open('CD_Pineda_opticthin.fits')
CD_Pineda_opticthin_w_header = add_header(CD_Pineda_opticthin)

### B. Mass Estimate
Again, equation can be imported but is included for convenience.


In [22]:
def CO13_or_C18O_Mass1(CD, Region):
    '''
    13CO mass estimate using Bourke et. al eqn. A11
    
    Parameters
    -------------
    CD: 2d array
        Column Density Map
    Region: pyregion object
        Ds9 Region
        
    Returns
    --------------
    Mass: float
        kg mass of region
    '''
    
    #Variables
    
    H_13CO_Conversion = 7e5
    Area = 1.44*10**32 #cm^2 #Area of a pixel in the image
    #Make distance variable ^^
    u_m = 4.5*10**(-27) #kg

    #Load in and Mask the CD of 12CO
    region1 = Region

    fdata = CD[0].data
    f = CD

    #1
    region2 = region1.as_imagecoord(f[0].header)

    mymask = region2.get_mask(hdu=f[0])

    real_mask = mymask * fdata

    #fits.writeto("13CO_CD_Masked.fits", real_mask)

    CO_13_masked = real_mask

    #Compute the mass per pixel via eqn. A11
    Mass_per_pixel = H_13CO_Conversion * Area* u_m *CO_13_masked 

    #Sum all the values in the 2D array, that will give the mass
    Mass = np.sum(Mass_per_pixel)

    return Mass


#### Bug #2
Pixel area is hard-coded. That's fine for M43, but not fine for extending to other regions, as the area depends on the distance. 

### B1. Mass Estimates with Column Density Method 1
We compute the mass estimates using the 3 region files from earlier and the first CD method

In [29]:
MassA1_R1 = CO13_or_C18O_Mass(CD_Pineda_w_header, region1)
np.format_float_scientific(MassA1_R1, 2)

'3.70e+33'

In [30]:
MassA1_R2 = CO13_or_C18O_Mass(CD_Pineda_w_header, region2)
np.format_float_scientific(MassA1_R2, 2)

'6.97e+33'

In [32]:
MassA1_R3 = CO13_or_C18O_Mass(CD_Pineda_w_header, region3)
np.format_float_scientific(MassA1_R3, 2)

'1.97e+33'

### B2. Mass Estimates with Column Density Method 2

In [33]:
MassA2_R1 = CO13_or_C18O_Mass(CD_Pineda_opticthin_w_header, region1)
np.format_float_scientific(MassA2_R1, 2)

'3.16e+33'

In [34]:
MassA2_R2 = CO13_or_C18O_Mass(CD_Pineda_opticthin_w_header, region2)
np.format_float_scientific(MassA2_R2, 2)

'5.86e+33'

In [35]:
MassA2_R3 = CO13_or_C18O_Mass(CD_Pineda_opticthin_w_header, region3)
np.format_float_scientific(MassA2_R3, 2)

'1.71e+33'